In [1]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


## 기본 정보
- 정보제공가능 지역별 공유전동킥보드 운영사 목록을 조회하는 기능
  - /getPMProvider
- 지역별 운영사명을 기반으로 탑승가능 공유전동킥보드 정보(운영사명, 장치ID, 베터리잔량, 좌표)를 조회하는 기능
  - /getPMListByProvider

In [2]:
# Python3 샘플 코드 #
import requests
import pandas as pd

# import keys     # key값 저장 파이썬 파일
# apikey = keys.key1_Decoding

# 일반 인증키 (Decoding) 사용
apikey = 'CTYRyVuDwSNng8WPJCY6Lm0A9JtQXyNRYn+7lI8FhtdmOgAF5g3dKjtS6AVkxJGU+dJXoZ2cmlEwbt7r6H1QsA=='

End_Point = 'http://apis.data.go.kr/1613000/PersonalMobilityInfoService'

# /getPMProvider
- 정보제공가능 지역별 공유전동킥보드 운영사 목록을 조회하는 기능

In [3]:
url = End_Point + '/getPMProvider'
params ={'serviceKey' : apikey,         # 공공데이터포털에서 받은 인증키
         'pageNo' : '1',                # 페이지번호
         'numOfRows' : '10',            # 한 페이지 결과 수
         '_type' : 'json',              # 데이터 타입(xml, json)
         'providerName' : '',           # 운영사명
         'cityCode': ''}                # 지역번호('지역별 공유전동킥보드 운영사 목록 조회' API 에서 지역명 기반 지역번호 확인)

response = requests.get(url, params=params)

if response.status_code == 200:
    result = response.json()
    print(result)
else:
    print('실패 : ', response.status_code)

{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': {'item': [{'citycode': 12, 'cityname': '세종특별시', 'providername': 'SWING'}, {'citycode': 12, 'cityname': '세종특별시', 'providername': 'GBIKE'}, {'citycode': 12, 'cityname': '세종특별시', 'providername': 'ALPACA'}]}, 'numOfRows': 10, 'pageNo': 1, 'totalCount': 3}}}


In [4]:
# items 추출
items = result['response']['body']['items']['item']

# DataFrame 생성
df_getPMProvider = pd.DataFrame(items)[['citycode', 'cityname', 'providername']]

# DataFrame 출력
df_getPMProvider

,citycode,cityname,providername
0,12,세종특별시,SWING
1,12,세종특별시,GBIKE
2,12,세종특별시,ALPACA


# /getPMListByProvider
- 지역별 운영사명을 기반으로 탑승가능 공유전동킥보드 정보(운영사명, 장치ID, 베터리잔량, 좌표)를 조회하는 기능

In [5]:
url = End_Point + '/getPMListByProvider'

df_sum = pd.DataFrame()

# iterrows()를 사용한 반복문
for index, row in df_getPMProvider.iterrows():
  params ={'serviceKey' : apikey,         # 공공데이터포털에서 받은 인증키
          'pageNo' : '1',                # 페이지번호
          'numOfRows' : '2000',            # 한 페이지 결과 수
          '_type' : 'json',              # 데이터 타입(xml, json)
          'providerName' : row['providername'],           # 운영사명
          'cityCode': '12'}                # 지역번호('지역별 공유전동킥보드 운영사 목록 조회' API 에서 지역명 기반 지역번호 확인)

  response = requests.get(url, params=params)

  if response.status_code == 200:
      result = response.json()
      print(result)

      items = result['response']['body']['items']['item']
      df_sum = pd.concat([df_sum, pd.DataFrame(items)[['battery', 'citycode', 'cityname', 'latitude', 'longitude', 'providername', 'vehicleid']]])
  else:
      print('실패 : ', response.status_code)

{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': {'item': [{'battery': 25, 'citycode': 12, 'cityname': '세종특별시', 'latitude': 36.09207, 'longitude': 128.445125, 'providername': 'SWING', 'vehicleid': 'cl6bj0mr3018ub6pkecix9bd8'}, {'battery': 99, 'citycode': 12, 'cityname': '세종특별시', 'latitude': 36.291605, 'longitude': 127.339351666667, 'providername': 'SWING', 'vehicleid': 'cl6a26rq900i4uipkg7o181xn'}, {'battery': 59, 'citycode': 12, 'cityname': '세종특별시', 'latitude': 36.34757833333333, 'longitude': 127.36936166666666, 'providername': 'SWING', 'vehicleid': 'cl6nhqowc00dzenpk642mfbdl'}, {'battery': 60, 'citycode': 12, 'cityname': '세종특별시', 'latitude': 36.469165, 'longitude': 127.273525, 'providername': 'SWING', 'vehicleid': 'cl5hqncir005qfgpkbyj869gr'}, {'battery': 77, 'citycode': 12, 'cityname': '세종특별시', 'latitude': 36.471605, 'longitude': 127.271645, 'providername': 'SWING', 'vehicleid': 'cl6a26p0p00e4uipk1jl3d9u2'}, {'battery': 26, 'citycode': 

In [6]:
# 공유자전거 대수
# SWING : 29
# GBIKE : 1522
# ALPACA : 238

In [7]:
df_sum

,battery,citycode,cityname,latitude,longitude,providername,vehicleid
0,25,12,세종특별시,36.092070,128.445125,SWING,cl6bj0mr3018ub6pkecix9bd8
1,99,12,세종특별시,36.291605,127.339352,SWING,cl6a26rq900i4uipkg7o181xn
2,59,12,세종특별시,36.347578,127.369362,SWING,cl6nhqowc00dzenpk642mfbdl
3,60,12,세종특별시,36.469165,127.273525,SWING,cl5hqncir005qfgpkbyj869gr
4,77,12,세종특별시,36.471605,127.271645,SWING,cl6a26p0p00e4uipk1jl3d9u2
...,...,...,...,...,...,...,...
229,75,12,세종특별시,36.634930,127.327840,ALPACA,424763
230,89,12,세종특별시,36.635914,127.317460,ALPACA,425727
231,62,12,세종특별시,36.636673,127.326390,ALPACA,424985
232,64,12,세종특별시,36.637444,127.324510,ALPACA,425011


In [8]:
df_sum['providername'].value_counts()

,count
providername,
GBIKE,1514
ALPACA,234
SWING,29


In [9]:
df_sum.to_csv('/gdrive/MyDrive/_Projects/재정데이터분석_미니프로젝트/DataSet/세종시_킥보드정보_202412012046.csv', index =False)

In [10]:
# SWING : 29
# GBIKE : 1522
# ALPACA : 238

df_SWING = df_sum[df_sum['providername'] == 'SWING']
df_GBIKE = df_sum[df_sum['providername'] == 'GBIKE']
df_ALPACA = df_sum[df_sum['providername'] == 'ALPACA']

In [11]:
print(df_SWING.shape, df_GBIKE.shape, df_ALPACA.shape)

(29, 7) (1514, 7) (234, 7)


In [23]:
df_kb = pd.read_csv('/gdrive/MyDrive/_Projects/재정데이터분석_미니프로젝트/DataSet/세종시_킥보드정보_202412012046.csv')

In [24]:
df_kb.shape

(1777, 7)

In [25]:
df_kb['providername'].value_counts()

,count
providername,
GBIKE,1514
ALPACA,234
SWING,29


In [26]:
df_kb_2 = df_kb.rename(columns = {'latitude':'위도', 'longitude' :'경도'})

In [27]:
df_kb_2.head(3)

,battery,citycode,cityname,위도,경도,providername,vehicleid
0,25,12,세종특별시,36.092070,128.445125,SWING,cl6bj0mr3018ub6pkecix9bd8
1,99,12,세종특별시,36.291605,127.339352,SWING,cl6a26rq900i4uipkg7o181xn
2,59,12,세종특별시,36.347578,127.369362,SWING,cl6nhqowc00dzenpk642mfbdl


In [28]:
df_kb_2_SWING = df_kb_2[df_kb_2['providername'] == 'SWING']
df_kb_2_GBIKE = df_kb_2[df_kb_2['providername'] == 'GBIKE']
df_kb_2_ALPACA = df_kb_2[df_kb_2['providername'] == 'ALPACA']

### folium 시각

In [29]:
import time
import folium

# 사용자 정의함수 경로 설정
import sys
# sys.path.append("G:/내 드라이브/Source/_Custom_Function")
sys.path.append("/gdrive/MyDrive/_Projects/재정데이터분석_미니프로젝트/Source/_Custom_Function")

# 사용자 정의함수 불러오기
import folium_function as fl_f

In [30]:
# center = [37.5665, 126.9780] # 서울시
center = [36.092070, 128.445125] # 세종시

kickboard_map = folium.Map(location=center, zoom_start=12, tiles='openstreetmap', name='OpenStreetMap')

# folium_CircleMarker : CircleMarker 표시하기
fl_f.folium_CircleMarker(_data = df_kb_2_SWING, _name = '킥보드 위치정보(SWING)', _map = kickboard_map, _show = True, _radius = 2, \
                    _color = 'purple', _fill = True, _fill_color = 'purple', _fill_opacity = 0.6, _popup = 'providername', tooltip = 'vehicleid')
fl_f.folium_CircleMarker(_data = df_kb_2_GBIKE, _name = '킥보드 위치정보(GBIKE)', _map = kickboard_map, _show = True, _radius = 2, \
                    _color = 'blue', _fill = True, _fill_color = 'blue', _fill_opacity = 0.6, _popup = 'providername', tooltip = 'vehicleid')
fl_f.folium_CircleMarker(_data = df_kb_2_ALPACA, _name = '킥보드 위치정보(ALPACA)', _map = kickboard_map, _show = True, _radius = 2, \
                    _color = 'red', _fill = True, _fill_color = 'red', _fill_opacity = 0.6, _popup = 'providername', tooltip = 'vehicleid')

#folium.TileLayer('openstreetmap', name='OpenStreetMap').add_to(bike_map)
folium.TileLayer('cartodbpositron', name='CartoDB Positron').add_to(kickboard_map)
folium.TileLayer('cartodbdark_matter', name='CartoDB Dark Matter').add_to(kickboard_map)

folium.LayerControl().add_to(kickboard_map)

# web에서 표시하기
# 지도 저장
kickboard_map.save('/gdrive/MyDrive/_Projects/재정데이터분석_미니프로젝트/Source/Source_두진/Source/folium_map.html')
# import webbrowser
# webbrowser.open('/gdrive/MyDrive/_Projects/재정데이터분석_미니프로젝트/Source/folium_map.html')